In [86]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [87]:
import re
import pandas as pd
from collections import defaultdict

def get_transaction_count(line):
    pattern = r"transactions = \{([^}]+)\}"
    match = re.search(pattern, line)
    if match:
        return set(re.findall(r"Transaction ([a-fA-F0-9]+)", match.group(1)))
    return set()

def get_timestamp(line):
    pattern = r"^\d+\.\d+"
    match = re.match(pattern, line)
    return float(match.group(0)) if match else None

def get_node_name(line):
    pattern = r"Node ([A-Za-z0-9]+)"
    match = re.search(pattern, line)
    return match.group(1) if match else None

def count_unique_mempool_transactions(file_path, node_number):
    unique_transactions = set()
    with open(file_path, 'r') as file:
        for line in file:
            if f"Node {node_number}" in line and "from mempool" in line:
                unique_transactions.update(re.findall(r"Transaction ([a-fA-F0-9]+)", line))
    return len(unique_transactions)

def extract_slot(message_line):
    """
    Extracts the slot number from an externalize message log line.
    Looks for the pattern "slot <number>".
    """
    pattern = r"slot (\d+)"
    match = re.search(pattern, message_line)
    if match:
        return int(match.group(1))
    return None

def process_log_lines(file_path):
    """
    Extracts **all** SCPExternalize messages per node and stores their relevant details.
    A new column 'Slot' is added by parsing the slot number from the log message.
    """
    data = []

    with open(file_path, 'r') as file:
        for line in file:
            # Only consider lines that contain relevant externalization messages
            if ("appended SCPExternalize message for slot" not in line and
                "adopting externalized value for slot" not in line):
                continue
            
            node_name = get_node_name(line)
            timestamp = get_timestamp(line)
            transactions = get_transaction_count(line)
            slot = extract_slot(line)  # Extract slot directly from the log line

            if node_name:
                data.append({
                    "node name": node_name,
                    "Timestamp of finalisation": timestamp,
                    "Finalised transactions": transactions,
                    "Externalize message": line.strip(),
                    "Slot": slot
                })
    
    # Convert the collected data to a DataFrame
    df = pd.DataFrame(data)
    
    # If any row didn't have a slot parsed, you might want to drop or handle it:
    df = df.dropna(subset=["Slot"])

    # Count the number of finalized transactions for each externalize message
    df["No. of finalised transactions"] = df["Finalised transactions"].apply(len)
    
    # Compute total transactions for each node from mempool logs
    df["total_transactions"] = df["node name"].apply(lambda node: count_unique_mempool_transactions(file_path, node))
    
    # Calculate number of transactions not finalized for each node
    df["no. of transactions not finalised"] = df["total_transactions"] - df["No. of finalised transactions"]

    return df

# Example usage:
# file_path = 'path/to/your/simulator_events_log.txt'
# df = process_log_lines(file_path)
# print(df)


In [88]:
file_path = 'src/simulator_events_log.txt'

df = process_log_lines(file_path)


# df_sorted = df.sort_values(by='Timestamp of finalisation', ascending=True)

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Prevent line wrapping
pd.set_option('display.colheader_justify', 'center')  # Center column headers
pd.set_option('display.float_format', '{:.4f}'.format)  # Format float values

df = df.sort_values(by='Slot', ascending=True)

display(df) 

,node name,Timestamp of finalisation,Finalised transactions,Externalize message,Slot,No. of finalised transactions,total_transactions,no. of transactions not finalised
0,7,8.6500,{3619a34a},8.65 - NODE - INFO - Node 7 appended SCPExtern...,1,1,118,117
32,35,9.0700,{3619a34a},9.07 - Node 35 adopting externalized value fo...,1,1,115,114
33,55,9.0700,{3619a34a},9.07 - Node 55 adopting externalized value fo...,1,1,118,117
34,17,9.0800,{3619a34a},9.08 - Node 17 adopting externalized value fo...,1,1,118,117
35,52,9.0800,{3619a34a},9.08 - Node 52 adopting externalized value fo...,1,1,109,108
36,58,9.0800,{3619a34a},9.08 - Node 58 adopting externalized value fo...,1,1,111,110
37,54,9.0900,{3619a34a},9.09 - Node 54 adopting externalized value fo...,1,1,109,108
38,24,9.1000,{3619a34a},9.10 - Node 24 adopting externalized value fo...,1,1,118,117
39,56,9.1000,{3619a34a},9.10 - Node 56 adopting externalized value fo...,1,1,124,123
40,1,9.1000,{3619a34a},9.10 - Node 1 adopting externalized value for...,1,1,195,194


## Analyze Transaction Matches across slots

In [89]:
def analyze_transaction_matches(df):
    tx_occurrences = {}

    for idx, row in df.iterrows():
        node = row['node name']
        slot = row['Slot']
        # row['Finalised transactions'] is a set; iterate through each transaction hash
        for tx in row['Finalised transactions']:
            if tx not in tx_occurrences:
                tx_occurrences[tx] = set()
            tx_occurrences[tx].add((node, slot))
    
    duplicates = {tx: occ for tx, occ in tx_occurrences.items() if len(occ) > 1} 

    return duplicates


duplicates = analyze_transaction_matches(df)
print("Transactions that appear in more than one slot:")
for tx, occ in duplicates.items():
    print(f"Transaction {tx} appears in: {sorted(list(occ))}")


Transactions that appear in more than one slot:
Transaction 3619a34a appears in: [('0', 1), ('1', 1), ('10', 1), ('11', 1), ('12', 1), ('13', 1), ('14', 1), ('15', 1), ('16', 1), ('17', 1), ('18', 1), ('19', 1), ('2', 1), ('20', 1), ('21', 1), ('22', 1), ('23', 1), ('24', 1), ('25', 1), ('26', 1), ('27', 1), ('28', 1), ('29', 1), ('3', 1), ('30', 1), ('31', 1), ('32', 1), ('33', 1), ('34', 1), ('35', 1), ('36', 1), ('37', 1), ('38', 1), ('39', 1), ('4', 1), ('40', 1), ('41', 1), ('42', 1), ('43', 1), ('44', 1), ('45', 1), ('46', 1), ('47', 1), ('48', 1), ('49', 1), ('5', 1), ('50', 1), ('51', 1), ('52', 1), ('53', 1), ('54', 1), ('55', 1), ('56', 1), ('57', 1), ('58', 1), ('59', 1), ('6', 1), ('7', 1), ('8', 1), ('9', 1)]
Transaction ecdc67cf appears in: [('0', 2), ('1', 2), ('10', 2), ('11', 2), ('12', 2), ('13', 2), ('14', 2), ('15', 2), ('16', 2), ('17', 2), ('18', 2), ('19', 2), ('2', 2), ('20', 2), ('21', 2), ('22', 2), ('23', 2), ('24', 2), ('25', 2), ('26', 2), ('27', 2), ('28',

#### SHOW ALL TRANSACTIONS THAT APPEAR IN MULTIPLE SLOTS FOR THE SAME NODE

In [90]:
def analyze_node_slots_and_multislot_occurrences(df):
    """
    Analyzes the DataFrame to extract:
      - For each node, a dictionary mapping each Slot to the set of finalized transaction hashes 
        that were reported in that slot.
      - For each node, a dictionary mapping transaction hashes (that occur in multiple slots) to the
        set of slot numbers where they appear.
    
    Args:
        df (pandas.DataFrame): A DataFrame with at least the columns:
            - 'node name'
            - 'Slot'
            - 'Finalised transactions' (a set of transaction hashes)
    
    Returns:
        A tuple (node_slots, multi_occurrences) where:
          - node_slots is a dict: { node_name: { slot: set(transaction hashes), ... }, ... }
          - multi_occurrences is a dict: { node_name: { transaction_hash: set(slots) where it appears, ... }, ... }
    """
    # Build a dictionary that records for each node which transactions are finalized in each slot.
    node_slots = {}
    for idx, row in df.iterrows():
        node = row['node name']
        slot = row['Slot']
        txs = row['Finalised transactions']  # expected to be a set of transaction hashes
        if node not in node_slots:
            node_slots[node] = {}
        if slot not in node_slots[node]:
            node_slots[node][slot] = set()
        node_slots[node][slot].update(txs)
    
    # Now, for each node, gather transaction occurrences across slots.
    multi_occurrences = {}
    for node, slot_dict in node_slots.items():
        tx_occurrences = {}  # maps tx -> set of slots where it occurs
        for slot, txs in slot_dict.items():
            for tx in txs:
                if tx not in tx_occurrences:
                    tx_occurrences[tx] = set()
                tx_occurrences[tx].add(slot)
        # Filter to only transactions that appear in more than one slot.
        multi = {tx: slots for tx, slots in tx_occurrences.items() if len(slots) > 1}
        if multi:
            multi_occurrences[node] = multi

    return node_slots, multi_occurrences

# Example usage:
node_slots, multi_occurrences = analyze_node_slots_and_multislot_occurrences(df)

print("Per-node transaction sets by slot:")
for node, slot_info in node_slots.items():
    print(f"Node {node}:")
    for slot, txs in sorted(slot_info.items()):
        print(f"  Slot {slot}: {sorted(list(txs))}")

print("\nTransactions that appear for the same node in multiple slots:")
for node, tx_info in multi_occurrences.items():
    print(f"Node {node}:")
    for tx, slots in tx_info.items():
        print(f"  Transaction {tx} appears in slots: {sorted(list(slots))}")


Per-node transaction sets by slot:
Node 7:
  Slot 1: ['3619a34a']
  Slot 2: ['2046889f', 'bac05fd5', 'ecdc67cf']
  Slot 3: ['43912236', '612d2204', '797f9dd', '844e021c', '9e2fafde', '9f71c598', 'a154ed8c', 'ae8c5cde', 'b0f9e8ea', 'bc15a2a4', 'd11ed90d', 'def737c9', 'f817e0c0', 'fb032cb2', 'fe84afbd']
  Slot 4: ['2fbfc115', '3c8afabd', '58dcc15f', '64444379', '6b0d72ab', '766f778e', '7e815674', '96ed0b34', '9b8f3a86', 'ae1835c5', 'aeef6ea3', 'afe73d8a', 'c78ed7d4', 'db08cbbc']
  Slot 5: ['1def4784', '202c53dd', '222c8a32', '29829300', '2ce932e9', '2dd12551', '449046b7', '850d8781', '889ab0d6', '8e649723', '95e50a93', 'a5940b7d', 'aacaa1f9', 'b80519cd', 'b9813244', 'bdf9af0e', 'c186207a', 'c8f7839b', 'cc178786', 'e38d1fe0', 'e5a9c14d']
  Slot 6: ['199cce86', '1ea3b0ba', '2471deeb', '5572b1c3', '56aa070e', '63265e47', '65413373', '6558600e', '6eb909d0', '83746e78', '8f3e11f2', '95f28e9a', '97c36b8f', 'a66d0b41', 'bf2a8c45', 'cc9f9fc8', 'd11f0619', 'e4bcda85', 'e7465551', 'ec9d8090', 'eca

In [91]:
import pandas as pd

def analyze_transaction_matches_with_msgs(df):
    """
    Find transactions that show up in more than one slot *and* record
    the first 3 words of the log messages that caused each appearance.
    
    Returns a dict:
      tx_hash -> {
         'occurrences': set of (node, slot),
         'msg_types':   set of first‑3‑word summaries
      }
    """
    tx_occurrences = {}   # tx -> set((node,slot))
    tx_msgtypes   = {}    # tx -> set(msg_type)
    
    for _, row in df.iterrows():
        node    = row['node name']
        slot    = row['Slot']
        txs     = row['Finalised transactions']
        msg     = row['Externalize message']
        # grab the first 3 words of the log line
        msg_type = " ".join(msg.split()[:10])
        
        for tx in txs:
            tx_occurrences.setdefault(tx, set()).add((node, slot))
            tx_msgtypes.setdefault(tx,   set()).add(msg_type)
    
    # now filter to only those that appear in >1 slot
    duplicates = {}
    for tx, occ in tx_occurrences.items():
        if len(occ) > 1:
            duplicates[tx] = {
                'occurrences': occ,
                'msg_types':   tx_msgtypes.get(tx, set())
            }
    return duplicates

# --- example usage ---
duplicates = analyze_transaction_matches_with_msgs(df)

print("Transactions that appear in more than one slot, with their first‐3‐word log types:")
for tx, info in duplicates.items():
    occ       = sorted(info['occurrences'])
    msg_types = sorted(info['msg_types'])
    print(f"- {tx}:")
    print(f"    slots:    {occ}")
    print(f"    messages: {msg_types}")


Transactions that appear in more than one slot, with their first‐3‐word log types:
- 3619a34a:
    slots:    [('0', 1), ('1', 1), ('10', 1), ('11', 1), ('12', 1), ('13', 1), ('14', 1), ('15', 1), ('16', 1), ('17', 1), ('18', 1), ('19', 1), ('2', 1), ('20', 1), ('21', 1), ('22', 1), ('23', 1), ('24', 1), ('25', 1), ('26', 1), ('27', 1), ('28', 1), ('29', 1), ('3', 1), ('30', 1), ('31', 1), ('32', 1), ('33', 1), ('34', 1), ('35', 1), ('36', 1), ('37', 1), ('38', 1), ('39', 1), ('4', 1), ('40', 1), ('41', 1), ('42', 1), ('43', 1), ('44', 1), ('45', 1), ('46', 1), ('47', 1), ('48', 1), ('49', 1), ('5', 1), ('50', 1), ('51', 1), ('52', 1), ('53', 1), ('54', 1), ('55', 1), ('56', 1), ('57', 1), ('58', 1), ('59', 1), ('6', 1), ('7', 1), ('8', 1), ('9', 1)]
    messages: ['8.65 - NODE - INFO - Node 7 appended SCPExternalize', '8.68 - Node 40 adopting externalized value for slot 1:', '8.77 - NODE - INFO - Node 4 appended SCPExternalize', '8.79 - Node 34 adopting externalized value for slot 1:',

In [92]:
def analyze_node_repeated_txs_with_msgs(df):
    """
    For each node, find transactions that are finalized
    in more than one slot.  For each such tx, record the
    slot *and* the first‑3‑words of the log message that
    reported it in that slot.
    
    Returns a dict:
      node -> {
        tx_hash -> [ (slot, msg_type), ... ]
      }
    """
    # intermediate store: node -> tx -> list of (slot,msg_type)
    temp = {}
    
    for _, row in df.iterrows():
        node = row['node name']
        slot = row['Slot']
        txs  = row['Finalised transactions']
        msg  = row['Externalize message']
        # first three words of the message
        msg_type = " ".join(msg.split()[:50])
        
        # initialize per‐node
        if node not in temp:
            temp[node] = {}
        
        for tx in txs:
            temp[node].setdefault(tx, []).append((slot, msg_type))
    
    # now filter to only those txes that appear in >1 distinct slot for each node
    result = {}
    for node, tx_dict in temp.items():
        repeated = {}
        for tx, slot_msgs in tx_dict.items():
            # collect distinct slots
            slots = { sm[0] for sm in slot_msgs }
            if len(slots) > 1:
                # keep all occurrences (slot,msg_type)
                repeated[tx] = slot_msgs
        if repeated:
            result[node] = repeated
    
    return result

# --- example usage ---
multi = analyze_node_repeated_txs_with_msgs(df)

for node, tx_info in multi.items():
    print(f"\nNode {node}:")
    for tx, occurrences in tx_info.items():
        print(f"  Transaction {tx!r} is in multiple slots:")
        for slot, msg_type in sorted(occurrences):
            print(f"    Slot {slot:>2} via “{msg_type}”")



Node 24:
  Transaction 'c186207a' is in multiple slots:
    Slot  4 via “53.94 - NODE - INFO - Node 24 appended SCPExternalize message for slot 4 to its storage and state, message = SCPExternalize(ballot=SCPBallot(counter=4, value=[Value, hash = 9210874228970330283, state = State.init, transactions = {[Transaction ae1835c5 time = 27.8690], [Transaction 6b0d72ab time = 33.1200], [Transaction 2fbfc115 time = 26.4328], [Transaction 29829300 time =”
    Slot  5 via “77.96 - Node 24 adopting externalized value for slot 5: [Value, hash = 464811589737039135, state = State.init, transactions = {[Transaction e38d1fe0 time = 49.2572], [Transaction 95e50a93 time = 50.5217], [Transaction 2ce932e9 time = 38.8084], [Transaction b80519cd time = 47.0321], [Transaction cc178786 time = 40.2349], [Transaction 29829300 time = 37.2111], [Transaction”
  Transaction 'e5a9c14d' is in multiple slots:
    Slot  4 via “53.94 - NODE - INFO - Node 24 appended SCPExternalize message for slot 4 to its storage and s

Analyse duplicates within transactions

In [93]:
def check_duplicates_within_slots(df):
    duplicates_within = []

    for idx, row in df.iterrows():
        node = row['node name']
        slot = row['Slot']
        tx_list = list(row['Finalised transactions'])
        seen = set()
        duplicates = set()

        for tx in tx_list:
            if tx in seen:
                duplicates.add(tx)
            else:
                seen.add(tx)

        if duplicates:
            duplicates_within.append({
                'node': node,
                'slot': slot,
                'duplicates': list(duplicates)
            })

    return duplicates_within

duplicates_within_slots = check_duplicates_within_slots(df)
print("Duplicates within each transaction set per slot:")
for entry in duplicates_within_slots:
    print(f"Node: {entry['node']}, Slot: {entry['slot']}, Duplicates: {entry['duplicates']}")


Duplicates within each transaction set per slot:


ADD INTERLEDGER CHECKS

In [94]:
def calculate_inter_ledger_agreement_time(df):
    df = df.sort_values(by='Timestamp of finalisation')
    time_diffs = df['Timestamp of finalisation'].diff().dropna()
    
    return time_diffs.mean()

avg_time = calculate_inter_ledger_agreement_time(df)
print(f"Average Inter-Ledger Agreement Time: {avg_time}")

Average Inter-Ledger Agreement Time: 0.35230055658627085


In [95]:
final_experiment_df = df[[
    "sequence number",
    "Timestamp of finalisation",
    "No. of finalised transactions",
    "no. of transactions not finalised"
]]

display(final_experiment_df)

KeyError: "['sequence number'] not in index"

In [ ]:
avg_difference = (final_experiment_df["no. of transactions not finalised"] - final_experiment_df["No. of finalised transactions"]).mean()

print(f"Average difference: {avg_difference}")

In [ ]:
avg_finalised = final_experiment_df["No. of finalised transactions"].mean()
avg_total = (final_experiment_df["No. of finalised transactions"] + 
             final_experiment_df["no. of transactions not finalised"]).mean()

finalised_percentage = (avg_finalised / avg_total) * 100 if avg_total != 0 else 0

print(f"Percentage of finalised transactions vs total: {finalised_percentage:.2f}%")